# Imports

In [29]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
import numpy as np
import seaborn as sns
from itertools import product
import random
from typing import List, Tuple

from reader import parse_csvs, SimulationResults

# Get Data

In [30]:
simulation_results: List[SimulationResults] = parse_csvs()

Started parsing...
Error parsing file Initial settings 182 - 09.05.2024 10-33-28.csv: [Errno 2] No such file or directory: 'c:\\Users\\PC\\Desktop\\studia\\Metody Systemowe i Decyzyjne\\Projekt\\MSiDProject\\data\\Attempt 182 - 09.05.2024 10-33-28.csv'
Ended parsing. Parsed files: 331, errors: 1


In [14]:
simulation_results

[SimulationResults(index=0, date=datetime.datetime(2024, 5, 9, 9, 41, 59), initial_parameters=    mapSize  startingCreatures  timeToSpawnAFood  hungerMult  speedMult  \
 0  23.76198                 15          4.529128        0.75          2   
 
    mitosisSpeedMult  mutationRange  bodyPartMutationChance  \
 0                 1       0.383709                0.005873   
 
    intervalBetweenBirths  hungerMultFromAge  ...  sensorGeneOn  \
 0               64.64482           0.003437  ...             1   
 
    processingGeneOn  gluttonyGeneOn  reproductionGeneOn  storageGeneOn  \
 0                 1               1                   1              1   
 
    parentEmpathyGeneOn  aggresivnessGeneOn  healthGeneOn  \
 0                    1                   1             1   
 
    foodPreferenceGeneOn  fightingPermitted  
 0                     0                  1  
 
 [1 rows x 30 columns], history=   simulation time  speed gene  sensor gene  processing gene  gluttony gene  \
 0      

# Cleaning Data